### **Before we start, remember we do not need to run on GPU because we are not training any model.**

In [ ]:
# GPU check (optional)
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## **Install YOLOv11**
Install the Ultralytics package, along with all required dependencies, in a Python environment (version 3.8 or higher) with PyTorch (version 1.8 or higher) using the following command: pip install ultralytics.

In [ ]:
# Install the Ultralytics YOLO package
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


Ultralytics 8.3.48 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/107.7 GB disk)


In [ ]:
# Verify the current working directory and create a data folder
import os
HOME = os.getcwd()
!mkdir -p {HOME}/data


### **Load Data**

In [ ]:
#Load the video to the Environment
%cd {HOME}
source_video = f"{HOME}/vehicle-counting.mp4"
display.clear_output()


Build a model
You can now choose the core model for your object tracking tasks; object detection, the corresponding checkpoints will be automatically downloaded.

In [ ]:
# Load the YOLO model
from ultralytics import YOLO

model = YOLO("yolo11n-seg.pt")


100%|██████████| 5.90M/5.90M [00:00<00:00, 54.4MB/s]


### **ByteTrack Installation through Roboflow Supervision**

# **Roboflow Supervision Installation**

In [ ]:
# Install supervision (used for visualization and tracking)
!pip install supervision

display.clear_output()


# **Import Required Library**

In [ ]:
# Import Required Libraries
import supervision as sv
import numpy as np


## **Initialization**

Limit the model to focus exclusively on the objects of interest. Additionally, since the goal is to count moving objects, you can define a specific region in the video—preferably closer to the camera—to track and count these objects. To achieve this, the region's coordinates should be specified.

In [ ]:
# Class names and IDs of interest (e.g., cars and trucks)
CLASS_NAMES_DICT = model.model.names
class_ids = [2, 7]

# Define counting line coordinates (adjusted based on video dimensions)
start = sv.Point(0, 1350)  # Starting point of the line
end = sv.Point(3840, 1350)  # Ending point of the line

# Target video path
target_video = f"{HOME}/result.mp4"


Analyse the input video.
And create a VideoInfo instance

In [ ]:
# Analyze the input video and create a VideoInfo instance
video_info = sv.VideoInfo.from_video_path(source_video)


Create ByteTrack instance.

In [ ]:
# Create ByteTrack instance for object tracking
byte_tracker = sv.ByteTrack(
    track_activation_threshold=0.25,
    lost_track_buffer=30,
    minimum_matching_threshold=0.8,
    frame_rate=30,
    minimum_consecutive_frames=3
)

byte_tracker.reset()

Connect YOLO to ByteTrack

In [ ]:
# Connect YOLO to ByteTrack

# Create a generator for video frames
generator = sv.get_video_frames_generator(source_video)

# Create LineZone instance for vehicle counting
line_zone = sv.LineZone(start=start, end=end)

# Create annotators for visualization
box_annotator = sv.BoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=1.5, text_color=sv.Color.BLACK)
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

# Define the callback function for processing each frame
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # YOLO model prediction and convert results to Supervision detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Filter detections to include only specified class IDs
    detections = detections[np.isin(detections.class_id, class_ids)]

    # Track the detections
    detections = byte_tracker.update_with_detections(detections)

    # Generate labels for visualization
    labels = [
        f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]

    # Annotate the frame
    annotated_frame = frame.copy()
    annotated_frame = trace_annotator.annotate(
        scene=annotated_frame, detections=detections
    )
    annotated_frame = box_annotator.annotate(
        scene=annotated_frame, detections=detections
    )
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels
    )

    # Update the LineZone for counting
    line_zone.trigger(detections)

    # Annotate the LineZone on the frame
    return line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

    %matplotlib inline
    show_frame_in_notebook(frame, (16, 16))


Process the video and save the final output

In [ ]:
#Process the video and save the output
sv.process_video(
    source_path = source_video,
    target_path = target_video,
    callback=callback
)